In [1]:
import networkx as nx
import numpy as np

def align_networks(X, Y):
    L = len(X)  # 获取层数
    aligned_Y = Y.copy()  # 创建Y的副本以进行对齐

    for l in range(L):  # 遍历每一层
        n = X[l].shape[0]  # 获取层l的单元数量

        # 创建S和R，分别表示网络X和Y的权重向量集合
        S = [tuple(weights) for weights in X[l]]
        R = [tuple(weights) for weights in Y[l]]  +  [tuple(-weights) for weights in Y[l]]
        
        # 创建二部图
        B = nx.Graph()
        B.add_nodes_from([(s, {'bipartite': 0}) for s in S])
        B.add_nodes_from([(r, {'bipartite': 1}) for r in R])
        
        threshold = 0.05  # 设置一个阈值来确定何时创建边
        B.add_edges_from([(s, r) for s in S for r in R if np.linalg.norm(np.array(s) - np.array(r)) <= threshold])


        # 计算最大二部匹配
        K = nx.bipartite.maximum_matching(B, top_nodes=set(S))
        
        # 处理匹配结果
        new_layer = np.zeros_like(aligned_Y[l])  # 创建一个新层来存储交换后的结果
        for i in range(n):
            matching_node = K.get((S[i]))
            if matching_node in R:
                idx_in_Y = R.index(matching_node) % aligned_Y[l].shape[0]
                new_layer[i] = aligned_Y[l][idx_in_Y]  # 更新新层的权重
                if tuple(-np.array(matching_node)) in R:  # 检查负权重匹配
                    new_layer[i] *= -1  # 反转该单元的输出
        
        aligned_Y[l] = new_layer  # 更新对齐后的网络Y的层
        
    return aligned_Y  # 返回对齐后的网络Y

# 使用您提供的X和Y测试align_networks函数
X = np.array([[[-0.5488135 , 0.71518937, 0.60276338, 0.54488318],
         [0.4236548 , 0.64589411, 0.43758721, 0.891773  ],
         [0.96366276, 0.38344152, 0.79172504, 0.52889492],
         [0.56804456, 0.92559664, 0.07103606, 0.0871293 ]]])
             
Y = np.array([[[-0.53921787, 0.72184176, 0.60832651, 0.55228343],
         [-0.43322717, -0.65187728, -0.4368168 , -0.89738358],
         [-0.95602825, -0.38623994, -0.7845921 , -0.5377883 ],
         [0.56848153, 0.92388988, 0.06632717, 0.09261397]]])
aligned_Y = align_networks(X, Y)
print(-aligned_Y)

[[[-0.53921787  0.72184176  0.60832651  0.55228343]
  [-0.43322717 -0.65187728 -0.4368168  -0.89738358]
  [-0.95602825 -0.38623994 -0.7845921  -0.5377883 ]
  [ 0.56848153  0.92388988  0.06632717  0.09261397]]]


In [2]:
X, Y 

(array([[[-0.5488135 ,  0.71518937,  0.60276338,  0.54488318],
         [ 0.4236548 ,  0.64589411,  0.43758721,  0.891773  ],
         [ 0.96366276,  0.38344152,  0.79172504,  0.52889492],
         [ 0.56804456,  0.92559664,  0.07103606,  0.0871293 ]]]),
 array([[[-0.53921787,  0.72184176,  0.60832651,  0.55228343],
         [-0.43322717, -0.65187728, -0.4368168 , -0.89738358],
         [-0.95602825, -0.38623994, -0.7845921 , -0.5377883 ],
         [ 0.56848153,  0.92388988,  0.06632717,  0.09261397]]]))